In [2]:
from openbb_terminal.sdk import openbb
import riskfolio as rp 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn 


In [3]:
start = "2023-01-22"
end = "2023-05-23"
symbols = ['AAPL','MSFT','AMZN','FTNT','BA','RTX','CVX','CAT','CMI','TXN','JPM','GEHC','LOW','AMT','CEG','ULTA','KURA']
tickers = openbb.stocks.ca.hist(symbols,start, end)

In [4]:
returns = tickers.pct_change()[1:]
returns.dropna(how="any", axis=1, inplace=True)

In [5]:
port = rp.Portfolio(returns=returns)

port.assets_stats(method_mu='hist', method_cov='hist', d=0.94)

port.lowerret = 0.0008
w_rp_c = port.rp_optimization(
    model="Classic",
    rm="MV",
    hist=True,
    rf=0,
    b=None
)

In [6]:
port_val = 16_000
w_rp_c["invest_amt"] = w_rp_c * port_val



In [7]:
dollar_amt = w_rp_c

dollar_amt['last_price'] = tickers.iloc[-1]
allocations = dollar_amt

In [8]:
allocations['shares'] = (allocations.invest_amt / allocations.last_price).astype(int)

allocations.index.name = 'Ticker'

allocations.sort_values (by='weights', ascending =False)


,weights,invest_amt,last_price,shares
Ticker,,,,
MSFT,0.099865,1597.845778,315.260010,5
AAPL,0.080199,1283.178277,171.559998,7
ULTA,0.077580,1241.274003,480.609985,2
FTNT,0.075466,1207.450582,68.089996,17
RTX,0.072149,1154.377457,94.080002,12
GEHC,0.071119,1137.906222,77.190002,14
KURA,0.058064,929.031741,13.930000,66
JPM,0.055180,882.887200,136.589996,6
CEG,0.053756,860.095336,84.040001,10


In [9]:
stocks = {'Ticker':['AAPL','MSFT','AMZN','FTNT','BA','RTX','CVX','CAT','CMI','TXN','JPM','GEHC','LOW','AMT','CEG','ULTA','KURA'],
          'Current_Value': [1126.0,1263.0,1057.0,1199.0,894.0,894.0,848.0,764.0,785.0,912.0,934.0,1013.0,950.0,794.0,940.0,901.0,989.0]
}

df=pd.DataFrame(stocks)

current_stocks = df.set_index('Ticker')

current_stocks.sort_values(by='Current_Value', ascending=False)

,Current_Value
Ticker,
MSFT,1263.0
FTNT,1199.0
AAPL,1126.0
AMZN,1057.0
GEHC,1013.0
KURA,989.0
LOW,950.0
CEG,940.0
JPM,934.0


In [10]:
mapping = pd.merge(allocations,current_stocks,on='Ticker')

mapping.astype('float').dtypes

mapping['diff'] = mapping['invest_amt'] - mapping['Current_Value']

mapping.sort_values (by='weights', ascending =False)

,weights,invest_amt,last_price,shares,Current_Value,diff
Ticker,,,,,,
MSFT,0.099865,1597.845778,315.260010,5,1263.0,334.845778
AAPL,0.080199,1283.178277,171.559998,7,1126.0,157.178277
ULTA,0.077580,1241.274003,480.609985,2,901.0,340.274003
FTNT,0.075466,1207.450582,68.089996,17,1199.0,8.450582
RTX,0.072149,1154.377457,94.080002,12,894.0,260.377457
GEHC,0.071119,1137.906222,77.190002,14,1013.0,124.906222
KURA,0.058064,929.031741,13.930000,66,989.0,-59.968259
JPM,0.055180,882.887200,136.589996,6,934.0,-51.112800
CEG,0.053756,860.095336,84.040001,10,940.0,-79.904664
